In [1]:
import numpy as np
import glob
import sys
sys.path.append("../Acdc/")
import Acdc 


# Create dictionaries for loading configurations nicely
### Also define some filepaths that should be adjusted based on your environment

In [3]:
#Structure of this dict:
#key: acdc number
#value: dict with the following keys
#   obj: Acdc object itself
#   config: configuration file path 
#   infiles: list of input data files for that board

#this set of boards is used in the July 2024 proton data at test-beam
acdcs = {44:{"obj":None, "config":"../configs/acdc44.yml", "infiles": [], "pedfiles": []}, \
	43:{"obj":None, "config":"../configs/acdc43.yml", "infiles": [], "pedfiles": []}}

#Configure some data filepaths
#you'll likely want to keep paths as we may be committing/pushing
#multiple paths. Just use a comment
datadir = "../../data/20240701ProtonDataFTBF/"


In [ ]:
#load the waveforms into the events attribute
for acdc_num, a in acdcs.items():
	a["obj"] = Acdc.Acdc(a["config"])

	#the configuration is now parsed, so we can find which station id
	#in order to parse the filetag at the end of the data filenames. 
	bnum = a["obj"].c["station_id"]
	#find all the data files for this board
	a["infiles"] = glob.glob(datadir + f"Raw_Proton*b{bnum}.txt")
	#find all the pedestal files for this board
	a["pedfiles"] = glob.glob(datadir + f"Raw_test*b{bnum}.txt")


	#go file by file and save pre-reduced data for each file.
	#The machinery in the ACDC class can instead handle all files,
	# it is so resource and RAM intensive that I instead opt to save
	#a pre-reduced output file for each input file, looping individually. 
	for f in a["infiles"]:
		#the function can take a list of files, 
		#so I pass just the one we are working on
		a["obj"].load_raw_data_to_events([f]) 

		#loads root file containing linearity data, 
		#calibrates pedestals based on ADC/voltage and does
		#pedestal subtraction, and loads timebase calibration.
		#Within these pedfiles, it finds the closest file to our
		#events without using a future pedfile. 
		a["obj"].calibrate_waveforms(a["pedfiles"])
		a["obj"].write_events_to_file(f.replace(".txt", "_prereduced.p"))




Loading 1 of 1 files
Loading timebase data
Loading pedestal data for file ../../data/20240701ProtonDataFTBF/Raw_test_20240627_134444_b0.txt
Loading linearity data
Calibrating pedestal data to convert to mV
Pedestal subtracting and converting to mV for all event waveforms
On event 0 of 454
On event 1 of 454
On event 2 of 454
On event 3 of 454
On event 4 of 454
On event 5 of 454
On event 6 of 454
On event 7 of 454
On event 8 of 454
On event 9 of 454
On event 10 of 454
On event 11 of 454
On event 12 of 454
On event 13 of 454
On event 14 of 454
On event 15 of 454
On event 16 of 454
On event 17 of 454
On event 18 of 454
On event 19 of 454
On event 20 of 454
On event 21 of 454
On event 22 of 454
On event 23 of 454
On event 24 of 454
On event 25 of 454
On event 26 of 454
On event 27 of 454
On event 28 of 454
On event 29 of 454
On event 30 of 454
On event 31 of 454
On event 32 of 454
On event 33 of 454
On event 34 of 454
On event 35 of 454
On event 36 of 454
On event 37 of 454
On event 38 of 4